# Import data& libraries 

In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support

In [194]:
path= "/mnt/d/Data/"

X_train = pd.read_csv(os.path.join(path, "train.csv"))
y = X_train["N_category"]
X_train = X_train[X_train.columns[:-1]]
X_train = X_train[X_train.columns[3:]]    #ID, 이미지 경로 제거
X_test = pd.read_csv(os.path.join(path, "test.csv"))
submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

In [195]:
X_train.head(5)

,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,63,2015-10-23,1,2,1,19.0,2.0,1.0,2.0,2.0,...,1.0,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN
1,51,2015-10-28,1,1,1,22.0,3.0,3.0,3.0,3.0,...,0.0,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN
2,37,2015-10-29,1,2,1,NaN,2.0,NaN,NaN,NaN,...,1.0,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0
3,54,2016-03-08,1,2,1,0.0,3.0,3.0,3.0,3.0,...,0.0,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN
4,57,2015-10-30,1,2,1,8.0,2.0,2.0,3.0,2.0,...,1.0,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN


In [196]:
print(X_train.info())
print(f"\nSize of X_train: {len(X_train)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   나이                 1000 non-null   int64  
 1   수술연월일              1000 non-null   object 
 2   진단명                1000 non-null   int64  
 3   암의 위치              1000 non-null   int64  
 4   암의 개수              1000 non-null   int64  
 5   암의 장경              931 non-null    float64
 6   NG                 949 non-null    float64
 7   HG                 914 non-null    float64
 8   HG_score_1         908 non-null    float64
 9   HG_score_2         908 non-null    float64
 10  HG_score_3         911 non-null    float64
 11  DCIS_or_LCIS_여부    1000 non-null   int64  
 12  DCIS_or_LCIS_type  126 non-null    float64
 13  T_category         996 non-null    float64
 14  ER                 999 non-null    float64
 15  ER_Allred_score    703 non-null    float64
 16  PR                 999 no

# Fill missing values

In [197]:
X_train = X_train.fillna(X_train.mean())

/tmp/ipykernel_360/3112579809.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())


# Process categorical values

In [198]:
X_train["수술연월일"].head(5)

0    2015-10-23
1    2015-10-28
2    2015-10-29
3    2016-03-08
4    2015-10-30
Name: 수술연월일, dtype: object

In [200]:
X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])

In [201]:
X_train["year"] = X_train["수술연월일"].dt.year
X_train["month"] = X_train["수술연월일"].dt.month
X_train["day"] = X_train["수술연월일"].dt.day
X_train["day_name"] = X_train["수술연월일"].dt.day_name()
date_list=[]
for i in range(len(X_train)):
    date_list.append(str(X_train["year"][i]).zfill(2)+str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))
X_train["수술연월일"] = date_list

encoder = LabelEncoder()
X_train["day_name"] = pd.DataFrame(encoder.fit_transform(np.reshape(np.array(X_train["day_name"]), (-1,1))))

index = [i for i in range(28)if i != 24]
X_train = X_train[X_train.columns[index]]

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [202]:
X_train["암의 장경"] = X_train["암의 장경"].fillna(X_train.groupby("T_category")["암의 장경"].transform("median"))
# X_train = original_X.copy()
X_train["암의 장경"] = X_train["암의 장경"]
# process_score(temp_X)

In [203]:
X_train

,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,month,day,day_name
0,63,20151023,1,2,1,19.000000,2.0,1.000000,2.000000,2.000000,...,6.00000,12.000000,0.0,1.0,0.129555,2.460457,0.145455,10,23,0
1,51,20151028,1,1,1,22.000000,3.0,3.000000,3.000000,3.000000,...,6.04936,70.000000,0.0,0.0,0.129555,2.460457,0.145455,10,28,4
2,37,20151029,1,2,1,18.375564,2.0,1.897155,2.588106,2.144273,...,4.00000,7.000000,0.0,1.0,0.129555,2.460457,0.000000,10,29,2
3,54,20160308,1,2,1,0.000000,3.0,3.000000,3.000000,3.000000,...,6.04936,1.000000,1.0,3.0,0.129555,2.460457,0.145455,3,8,3
4,57,20151030,1,2,1,8.000000,2.0,2.000000,3.000000,2.000000,...,6.04936,8.000000,1.0,2.0,1.000000,5.440000,0.145455,10,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,65,20061222,1,2,1,25.000000,1.0,1.000000,2.000000,1.000000,...,6.04936,18.037229,0.0,0.0,0.129555,2.460457,0.145455,12,22,0
996,48,20061117,1,1,1,7.000000,1.0,1.000000,1.000000,1.000000,...,6.04936,18.037229,0.0,0.0,0.129555,2.460457,0.145455,11,17,0
997,64,20061110,1,2,1,15.000000,1.0,1.000000,2.000000,1.000000,...,6.04936,18.037229,0.0,0.0,0.129555,2.460457,0.145455,11,10,0
998,50,20060922,1,1,1,7.000000,1.0,1.000000,1.000000,2.000000,...,6.04936,18.037229,0.0,0.0,0.129555,2.460457,0.145455,9,22,0


In [204]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   나이                 1000 non-null   int64  
 1   수술연월일              1000 non-null   object 
 2   진단명                1000 non-null   int64  
 3   암의 위치              1000 non-null   int64  
 4   암의 개수              1000 non-null   int64  
 5   암의 장경              1000 non-null   float64
 6   NG                 1000 non-null   float64
 7   HG                 1000 non-null   float64
 8   HG_score_1         1000 non-null   float64
 9   HG_score_2         1000 non-null   float64
 10  HG_score_3         1000 non-null   float64
 11  DCIS_or_LCIS_여부    1000 non-null   int64  
 12  DCIS_or_LCIS_type  1000 non-null   float64
 13  T_category         1000 non-null   float64
 14  ER                 1000 non-null   float64
 15  ER_Allred_score    1000 non-null   float64
 16  PR                 1000 n

In [205]:
from sklearn.model_selection import train_test_split

In [206]:
original_X = X_train.copy()

In [207]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, stratify=y, test_size=0.2)

# Train vanilla RF 

In [208]:
from sklearn.ensemble import RandomForestClassifier

In [209]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [210]:
model.score(X_valid, y_valid)

0.815

In [211]:
X_test

,ID,img_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,BC_01_0011,./test_imgs/BC_01_0011.png,55,2015-11-17,2,2,1,23.0,2.0,2.0,...,1.0,8.0,0.0,2.0,5.00,0.0,1.0,NaN,NaN,NaN
1,BC_01_0220,./test_imgs/BC_01_0220.png,43,2020-06-09,4,2,1,13.0,3.0,2.0,...,1.0,4.0,1.0,8.0,8.67,0.0,0.0,NaN,NaN,NaN
2,BC_01_0233,./test_imgs/BC_01_0233.png,76,2020-05-14,1,1,1,NaN,NaN,NaN,...,1.0,6.0,1.0,6.0,NaN,NaN,2.0,NaN,NaN,NaN
3,BC_01_0258,./test_imgs/BC_01_0258.png,58,2020-05-20,1,2,1,1.3,2.0,2.0,...,1.0,7.0,0.0,NaN,21.17,1.0,3.0,NaN,NaN,0.0
4,BC_01_0260,./test_imgs/BC_01_0260.png,56,2020-05-20,1,2,2,15.0,3.0,3.0,...,1.0,8.0,1.0,3.0,20.57,1.0,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,BC_01_3328,./test_imgs/BC_01_3328.png,61,2009-10-30,1,1,1,18.0,2.0,2.0,...,1.0,5.0,1.0,3.0,NaN,0.0,0.0,NaN,NaN,NaN
246,BC_01_3404,./test_imgs/BC_01_3404.png,42,2009-05-19,1,2,1,20.0,1.0,2.0,...,1.0,5.0,1.0,5.0,NaN,0.0,1.0,NaN,NaN,NaN
247,BC_01_3418,./test_imgs/BC_01_3418.png,37,2009-04-24,1,1,1,17.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN
248,BC_01_3438,./test_imgs/BC_01_3438.png,37,2009-02-06,1,1,1,7.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN


In [212]:
def preprocess(X_train):
    X_train = X_train[X_train.columns[2:]]
    X_train = X_train.fillna(X_train.mean())    
    X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])
    X_train["year"] = X_train["수술연월일"].dt.year
    X_train["month"] = X_train["수술연월일"].dt.month
    X_train["day"] = X_train["수술연월일"].dt.day
    X_train["day_name"] = X_train["수술연월일"].dt.day_name()

    date_list=[]
    for i in range(len(X_train)):
        date_list.append(str(X_train["year"][i]).zfill(2)+str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

    X_train["수술연월일"] = date_list

    encoder = LabelEncoder()
    X_train["day_name"] = pd.DataFrame(encoder.fit_transform(np.reshape(np.array(X_train["day_name"]), (-1,1))))

    index = [i for i in range(28)if i != 24]
    X_train = X_train[X_train.columns[index]]
    return X_train

In [213]:
X_test = preprocess(X_test)

/tmp/ipykernel_360/2787174443.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [214]:
X_test

,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,month,day,day_name
0,55,20151117,2,2,1,23.000000,2.000000,2.000000,3.000000,2.000000,...,2.0,5.000000,0.00000,1.0,0.213115,2.290222,0.0,11,17,4
1,43,20200609,4,2,1,13.000000,3.000000,2.000000,2.000000,3.000000,...,8.0,8.670000,0.00000,0.0,0.213115,2.290222,0.0,6,9,4
2,76,20200514,1,1,1,18.793249,2.076596,1.974359,2.659483,2.155172,...,6.0,17.404697,0.27459,2.0,0.213115,2.290222,0.0,5,14,3
3,58,20200520,1,2,1,1.300000,2.000000,2.000000,3.000000,2.000000,...,6.0,21.170000,1.00000,3.0,0.213115,2.290222,0.0,5,20,5
4,56,20200520,1,2,2,15.000000,3.000000,3.000000,3.000000,3.000000,...,3.0,20.570000,1.00000,3.0,0.213115,2.290222,0.0,5,20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,61,20091030,1,1,1,18.000000,2.000000,2.000000,3.000000,2.000000,...,3.0,17.404697,0.00000,0.0,0.213115,2.290222,0.0,10,30,0
246,42,20090519,1,2,1,20.000000,1.000000,2.000000,3.000000,2.000000,...,5.0,17.404697,0.00000,1.0,0.213115,2.290222,0.0,5,19,4
247,37,20090424,1,1,1,17.000000,1.000000,1.000000,3.000000,1.000000,...,5.0,17.404697,0.00000,0.0,0.213115,2.290222,0.0,4,24,0
248,37,20090206,1,1,1,7.000000,1.000000,1.000000,2.000000,1.000000,...,5.0,17.404697,0.00000,0.0,0.213115,2.290222,0.0,2,6,0


In [228]:
pred = model.predict(X_test)

In [229]:
submission["N_category"] = pred

In [230]:
submission.to_csv("./submission_tabular.csv", index=False)

# Data preprocessing 

In [215]:
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

In [216]:
original_X["수술연월일"] = pd.to_datetime(original_X["수술연월일"])
original_X["year"] = original_X["수술연월일"].dt.year
original_X["month"] = original_X["수술연월일"].dt.month
original_X["day"] = original_X["수술연월일"].dt.day
original_X["day_name"] = original_X["수술연월일"].dt.day_name()
encoder = LabelEncoder()
original_X["day_name"] = pd.DataFrame(encoder.fit_transform(np.reshape(np.array(original_X["day_name"]), (-1,1))))

date_list=[]
for i in range(len(original_X)):
    date_list.append(str(original_X["year"][i]).zfill(2)+str(original_X["month"][i]).zfill(2) +str(original_X["day"][i]).zfill(2))
original_X["수술연월일"] = date_list

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [217]:
def process_score(X):
    f1_list=[]
#     X_train, X_valid, y_train, y_valid = train_test_split(data, y, stratify=y, test_size=0.2)
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(X, y)    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        f1_list.append(precision_recall_fscore_support(pred, y_test, average="macro")[2])
    print(np.array(f1_list).mean())

In [218]:
process_score(original_X)

0.5515236378569701


/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [219]:
feature_list=[]
for i in range(len(model.feature_importances_)):
    feature_list.append([X_train.columns[i], model.feature_importances_[i]])
feature_df = pd.DataFrame(feature_list)
feature_df.sort_values(1, ascending=False)
# feature_list = np.array(feature_list)
# plt.xticks(rotation=45)
# plt.bar(feature_list[:,0], feature_list[:,1].astype("float"))

,0,1
1,수술연월일,0.188094
5,암의 장경,0.085509
11,DCIS_or_LCIS_여부,0.071029
6,NG,0.064210
0,나이,0.062080
25,day,0.051184
18,KI-67_LI_percent,0.046831
24,month,0.040431
7,HG,0.039255
17,PR_Allred_score,0.035308


In [220]:
X = pd.read_csv(os.path.join(path, "train.csv"))
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 1000 non-null   object 
 1   img_path           1000 non-null   object 
 2   mask_path          1000 non-null   object 
 3   나이                 1000 non-null   int64  
 4   수술연월일              1000 non-null   object 
 5   진단명                1000 non-null   int64  
 6   암의 위치              1000 non-null   int64  
 7   암의 개수              1000 non-null   int64  
 8   암의 장경              931 non-null    float64
 9   NG                 949 non-null    float64
 10  HG                 914 non-null    float64
 11  HG_score_1         908 non-null    float64
 12  HG_score_2         908 non-null    float64
 13  HG_score_3         911 non-null    float64
 14  DCIS_or_LCIS_여부    1000 non-null   int64  
 15  DCIS_or_LCIS_type  126 non-null    float64
 16  T_category         996 no

In [221]:
corr = X.corr()

/tmp/ipykernel_360/3921787701.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = X.corr()


## missing values

In [222]:
print(corr["암의 장경"].sort_values(ascending=False))

암의 장경                1.000000
T_category           0.809469
HG                   0.353011
HER2_SISH_ratio      0.348833
NG                   0.311455
HG_score_2           0.309582
HG_score_3           0.308494
N_category           0.292102
HG_score_1           0.249808
KI-67_LI_percent     0.237337
나이                   0.127918
HER2_SISH            0.112631
DCIS_or_LCIS_type    0.095926
ER_Allred_score      0.080298
PR_Allred_score      0.062952
DCIS_or_LCIS_여부      0.021600
암의 개수                0.011457
진단명                  0.006628
HER2                -0.002667
암의 위치               -0.005704
HER2_IHC            -0.015271
BRCA_mutation       -0.078003
PR                  -0.101581
ER                  -0.140662
Name: 암의 장경, dtype: float64


암의 장경은 T_category및 HG와 연관성이 크므로 이를 이용하여 결측치를 채움

In [223]:
X.groupby("T_category").agg(["mean", "median"])["암의 장경"]

/tmp/ipykernel_360/142092162.py:1: FutureWarning: ['ID', 'img_path', 'mask_path', '수술연월일'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  X.groupby("T_category").agg(["mean", "median"])["암의 장경"]


,mean,median
T_category,,
0.0,0.000000,0.0
1.0,12.192583,13.0
2.0,27.715082,25.0
3.0,66.812500,60.0
4.0,60.750000,68.0


In [224]:
X.groupby("HG").agg(["mean", "median"])["암의 장경"]

/tmp/ipykernel_360/40916919.py:1: FutureWarning: ['ID', 'img_path', 'mask_path', '수술연월일'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  X.groupby("HG").agg(["mean", "median"])["암의 장경"]


,mean,median
HG,,
1.0,13.491135,12.0
2.0,18.653409,17.0
3.0,25.632979,22.0


In [225]:
process_score(original_X)

0.5653889500066233


/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [226]:
X["암의 장경"] = X["암의 장경"].fillna(X.groupby("T_category")["암의 장경"].transform("median"))
temp_X = original_X.copy()
temp_X["암의 장경"] = X["암의 장경"]
process_score(temp_X)

0.5660857120304175


/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [227]:
process_score(original_X[original_X.columns][])

SyntaxError: invalid syntax (2165959682.py, line 1)

In [ ]:
original_X[original_X.columns]

In [ ]:
original_X.columns

In [ ]:
index = [i for i in range(28)if i != 24]
del_year = original_X[original_X.columns[index]]

In [ ]:
process_score(original_X[original_X.columns][])